# jovian-ml-project-rough

Use the "Run" button to execute the code.

In [1]:
#pip install jovian --upgrade --quiet

In [2]:
#import jovian

In [3]:
# Execute this to save new versions of the notebook
#jovian.commit(project="jovian-ml-project",filename="jovian-ml-project")

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import xml.etree.ElementTree as ET
import time
import re
from tqdm import tqdm
%matplotlib inline

In [5]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import warnings
warnings.simplefilter(action='ignore')

In [6]:
tqdm.pandas(desc="my bar!")

<a id='top'>**Beginning of Notebook**</a>

[model train text data](#train_data)

#### train data

In [7]:
train_text_df = pd.read_csv('training_text.zip', sep="\|\|", encoding="utf-8",engine="python", skiprows=1, names=["ID", "Text"],compression='zip')

In [8]:
train_vari_df = pd.read_csv('training_variants.zip',compression='zip')

In [9]:
train_vari_df['Class'].value_counts()

7    953
4    686
1    568
2    452
6    275
5    242
3     89
9     37
8     19
Name: Class, dtype: int64

In [10]:
train_text_df.shape, train_vari_df.shape

((3321, 2), (3321, 4))

In [11]:
train_text_df.columns, train_vari_df.columns

(Index(['ID', 'Text'], dtype='object'),
 Index(['ID', 'Gene', 'Variation', 'Class'], dtype='object'))

In [12]:
# merging the text and variation data
train_df = pd.merge(train_vari_df,train_text_df,on='ID')

In [13]:
# taking a sample of the dataset
train_df = train_df.sample(frac=0.2)

In [14]:
train_df.head()

,ID,Gene,Variation,Class,Text
1508,1508,ALK,L1122V,7,"In the era of personalized medicine, understan..."
2815,2815,BRCA2,K2472T,6,Purpose Rare missense substitutions and in-fra...
2947,2947,KDR,D717V,7,Angiosarcomas (AS) represent a heterogeneous g...
1040,1040,TSC2,R1200W,4,Tuberous sclerosis complex (TSC) is an autosom...
534,534,SMAD2,R321Q,4,Abstract To investigate genetic alterations i...


In [15]:
train_df.shape

(664, 5)

In [16]:
train_df['Class'].value_counts()

7    196
4    147
1    117
2     93
6     49
5     37
3     16
9      5
8      4
Name: Class, dtype: int64

In [17]:
test_text_df = pd.read_csv('test_text.zip', sep="\|\|", encoding="utf-8",engine="python", skiprows=1, names=["ID", "Text"],compression='zip')


In [18]:
test_vari_df  = pd.read_csv('test_variants.zip',compression='zip')

In [19]:
test_vari_df.head()

,ID,Gene,Variation
0,0,ACSL4,R570S
1,1,NAGLU,P521L
2,2,PAH,L333F
3,3,ING1,A148D
4,4,TMEM216,G77A


In [20]:
test_vari_df.shape

(5668, 3)

In [21]:
# test_vari_df sample
test_vari_df=test_vari_df.sample(frac=0.1)

In [22]:
# test_text_df sample
test_text_df = test_text_df.sample(frac=0.1)

In [23]:
test_text_df.shape

(567, 2)

In [24]:
test_text_df.head()

,ID,Text
2066,2066,The DNA-damage response (DDR) plays a crucial ...
5358,5358,Mismatch-repair factors have a prominent role ...
436,436,Gain-of-function D816V point mutation within t...
5059,5059,Mutations in inositol polyphosphate 5-phosphat...
3114,3114,The androgen receptor (AR) is involved in the ...


In [25]:
train_vari_df.groupby('Gene').sum().head()

,ID,Class
Gene,,
ABL1,22087,62
ACVR1,3207,16
AGO2,10435,6
AKT1,37898,165
AKT2,15103,76


In [26]:
train_vari_df.groupby('Gene').count().head()

,ID,Variation,Class
Gene,,,
ABL1,26,26,26
ACVR1,3,3,3
AGO2,5,5,5
AKT1,28,28,28
AKT2,11,11,11


In [27]:
train_vari_df[train_vari_df['Gene'] == 'AGO2']

,ID,Gene,Variation,Class
2085,2085,AGO2,R812A,1
2086,2086,AGO2,R647A,1
2087,2087,AGO2,R583A,1
2088,2088,AGO2,Amplification,2
2089,2089,AGO2,Q545A,1


In [28]:
merged_df=train_vari_df.merge(train_text_df,on= 'ID')

[link to destination](#destination)

In [29]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [30]:
revised_punctuation='!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~'

In [31]:
revised_punctuation

'!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~'

In [32]:
#!pip install nltk

In [33]:
import nltk
nltk.download('words')
from nltk.corpus import words
words=nltk.corpus.words.words()

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\anubr\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [34]:
len(set(words))

235892

In [35]:
nltk_words=set(words)

In [36]:
# set did not help as it reduces the number to 
# def reduce_words(row,index): 
#     text=row[1]
#     if isinstance(text,str):
#         new_text_words = set(text.split())
        
#         important_words = new_text_words.difference(words)
#         edited_words = " ".join(important_words)
        
#         intersected_words = new_text_words.intersection(words)
#         edit_intersect_words = " ".join(intersected_words)
#         train_text_df['new_text'][index] = edited_words
#         train_text_df['intsct_text'][index] = edit_intersect_words 

In [37]:
#from collections import Counter

In [38]:
def reduce_words_df(row,index): 
    text=row[1]
    if isinstance(text,str):
        new_text_words = text.split()
        counted_words = Counter(new_text_words) 
        counted_words=sorted(counted_words.items(),key=lambda x:x[1], reverse=True)
        test['new_text'][index] = counted_words        

In [39]:
# for index,row in test.iterrows():
#     reduce_words(row,index)

In [40]:
def cleanse_data(data):        
    if isinstance(data,str):
        no_punctuation = [char for char in data.replace('\n',' ') if char not in revised_punctuation]
        no_punctuation_txt = ''.join(no_punctuation)                   
        return no_punctuation_txt
    else:
        return "NA"

In [41]:
def reduce_words(data):
    data = data.split()
    words = [word for word in data if word not in nltk_words]
    text = " ".join(words)
    return text    

In [42]:
def word_count(text):
    text_lst=text.split()
    vector = [text_lst.count(str(word)) for word in text_lst]
    return vector[:300]

In [43]:
from nltk.probability import FreqDist

we have obtained the frequency distribution of words using freqdist

In [44]:
def freq_dist(data):
    data=data.split(" ")
    freq=FreqDist(data) # gets word frequency
    freq=sorted(freq.items(),key=lambda x:x[1], reverse=True) # sorts words from highest to lowest frequency
    return [item[1] for item in freq][:300] # returns a fixed length of 300 words

In [45]:
# implementing bag of words

In [46]:
from sklearn.feature_extraction.text import CountVectorizer

In [47]:
bow_vectorizer = CountVectorizer(stop_words='english',max_features=1600)

In [48]:
# implementing tf-idf

In [49]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [50]:
tfidf_vectorizer = TfidfVectorizer(
    min_df=1, max_features=1600, strip_accents='unicode',lowercase =True,
    analyzer='word', token_pattern=r'\w+', ngram_range=(1, 3), use_idf=True, 
    smooth_idf=True, sublinear_tf=True, stop_words = 'english')

In [51]:
# https://melaniewalsh.github.io/Intro-Cultural-Analytics/05-Text-Analysis/03-TF-IDF-Scikit-Learn.html
# https://www.datacamp.com/community/tutorials/understanding-logistic-regression-python
# https://machinelearningmastery.com/multinomial-logistic-regression-with-python/
# https://www.kaggle.com/elemento/personalizedmedicine-rct

In [52]:
from sklearn.linear_model import LogisticRegression

In [53]:
from sklearn.metrics import log_loss

In [54]:
# define the multinomial logistic regression model
bow_model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
tfidf_model = LogisticRegression(multi_class='multinomial', solver='lbfgs')

In [55]:
#sorted(bow_model.coef_.tolist()[0])[:5]

In [56]:
#sorted(tfidf_model.coef_.tolist()[0])[:5]

<a id="train_data" style="font-size:22px">model train text sample data</a>

[Beginning of Notebook](#top)

### handling the train text sample data                  

#### text preprocessing

In [57]:
train_df.head()

,ID,Gene,Variation,Class,Text
1508,1508,ALK,L1122V,7,"In the era of personalized medicine, understan..."
2815,2815,BRCA2,K2472T,6,Purpose Rare missense substitutions and in-fra...
2947,2947,KDR,D717V,7,Angiosarcomas (AS) represent a heterogeneous g...
1040,1040,TSC2,R1200W,4,Tuberous sclerosis complex (TSC) is an autosom...
534,534,SMAD2,R321Q,4,Abstract To investigate genetic alterations i...


In [58]:
# remove punctuation
train_df['cleaned_text'] = train_df['Text'].map(cleanse_data,na_action='ignore')

In [59]:
# filtering all common english words
train_df['cleaned_text'] = train_df['cleaned_text'].map(reduce_words,na_action='ignore')

In [60]:
train_df['Text_len'] = train_df['Text'].map(lambda x:len(str(x)))
train_df['cleaned_text_len'] = train_df['cleaned_text'].map(lambda x:len(str(x)))

In [61]:
train_df.head()

,ID,Gene,Variation,Class,Text,cleaned_text,Text_len,cleaned_text_len
1508,1508,ALK,L1122V,7,"In the era of personalized medicine, understan...",In personalized drivers oncogenesis characteri...,46112,21449
2815,2815,BRCA2,K2472T,6,Purpose Rare missense substitutions and in-fra...,Purpose Rare missense substitutions in-frame d...,22678,8114
2947,2947,KDR,D717V,7,Angiosarcomas (AS) represent a heterogeneous g...,Angiosarcomas AS tumors occurring locations se...,15704,6686
1040,1040,TSC2,R1200W,4,Tuberous sclerosis complex (TSC) is an autosom...,Tuberous TSC characterized symptoms hamartomat...,21240,10075
534,534,SMAD2,R321Q,4,Abstract To investigate genetic alterations i...,Abstract To alterations TGF-β signaling colore...,67598,27850


In [62]:
train_df[train_df['cleaned_text'].map(lambda x:not isinstance(x,str))]

,ID,Gene,Variation,Class,Text,cleaned_text,Text_len,cleaned_text_len


In [ ]:
train_df.drop([1109],axis=0,inplace=True)

#### converting text to numbers

In [63]:
train_df_corpus = train_df['cleaned_text'].tolist()

In [64]:
bow_vectorizer.fit(train_df_corpus)

CountVectorizer(max_features=1600, stop_words='english')

In [66]:
train_df_counts= bow_vectorizer.transform(train_df_corpus)

In [67]:
print("Vocabulary: ", bow_vectorizer.vocabulary_)

Vocabulary:  {'drivers': 545, 'oncogenesis': 1078, 'characteristics': 377, 'therapies': 1484, 'identifying': 782, 'mutations': 1018, 'sequencing': 1355, 'cohorts': 401, 'family': 626, 'including': 812, 'genes': 695, 'oncogenes': 1077, 'forms': 665, 'changes': 376, 'implicated': 805, 'et': 593, 'mycn': 1020, 'oncogene': 1076, 'tumors': 1525, 'subsets': 1433, 'encoding': 571, 'like': 902, 'families': 625, 'reviewed': 1305, 'old': 1074, 'pedigrees': 1127, '20': 64, 'showed': 1361, 'using': 1543, 'genome': 697, 'polymorphisms': 1165, 'snps': 1397, 'identified': 781, 'germline': 703, 'alk': 238, 'predicted': 1177, 'substitutions': 1434, 'fig': 645, 'proteins': 1210, 'containing': 432, 'indicated': 817, 'figure': 647, 'domains': 536, 'pathways': 1110, 'inhibitors': 831, 'amino': 249, 'acid': 208, 'positions': 1169, 'snp': 1396, 'based': 305, 'methods': 974, 'studies': 1428, 'results': 1298, 'resulted': 1297, 'increases': 814, 'autophosphorylation': 298, 'akt': 229, 'lines': 905, '3t3': 119, 

In [68]:
X_train_bow = train_df_counts.toarray()

In [69]:
y_train = train_df['Class']

#### model fitting

In [70]:
train_df.shape

(664, 8)

In [71]:
from numpy import random

In [72]:
# create a dummy y value for getting the baseline accuracy
y_dummy = random.rand(664,9)

In [ ]:
#y_dummy

In [73]:
# baseline loss metric value
log_loss(y_train,y_dummy)

2.4584973576484925

#### Logistic Regression Model

In [74]:
bow_model.fit(X_train_bow,y_train)

LogisticRegression(multi_class='multinomial')

In [75]:
sorted(bow_model.coef_.tolist()[0])[:5]

[-0.22804384844291725,
 -0.19162836814100284,
 -0.18586572686654063,
 -0.16930186716798076,
 -0.16458155697687452]

In [76]:
y_pred_train_bow = bow_model.predict_proba(X_train_bow)

In [77]:
log_loss(y_train,y_pred_train_bow)

0.13377678273804283

In [78]:
tfidf_vectorizer.fit(train_df_corpus)

TfidfVectorizer(max_features=1600, ngram_range=(1, 3), stop_words='english',
                strip_accents='unicode', sublinear_tf=True,
                token_pattern='\\w+')

In [79]:
X_train_tfidf = tfidf_vectorizer.transform(train_df_corpus)

In [80]:
X_train_tfidf = X_train_tfidf.toarray()

In [81]:
tfidf_model.fit(X_train_tfidf,y_train)

LogisticRegression(multi_class='multinomial')

In [82]:
y_pred_train_tfidf = tfidf_model.predict_proba(X_train_tfidf)

In [83]:
log_loss(y_train,y_pred_train_tfidf)

1.0410624198183753

#### handling the test text sample data

In [84]:
#test_text_df.sort_values(['ID']).head()

In [85]:
#test_text_df.shape

In [86]:
#test_text_df['cleaned_text'] = test_text_df['Text'].map(cleanse_data,na_action='ignore')

In [87]:
#test_text_df['cleaned_text'] = test_text_df['cleaned_text'].map(reduce_words)

In [88]:
#test_text_df['cleaned_text_vector'] = test_text_df['cleaned_text'].map(word_count)

In [89]:
#test_text_df['sorted_cleaned_text_vector'] = test_text_df['cleaned_text'].progress_map(freq_dist)

In [90]:
#test_text_df = test_text_df.drop(['Text'],axis=1)

In [91]:
#test_text_df.head()

In [92]:
# get the text samples as list
#test_corpus = test_text_df['cleaned_text'].tolist()

In [93]:
# transform the test case to get bag of words
#test_bag_of_words_train = bow_vectorizer.fit_transform(test_corpus)

In [94]:
# convert the dictionary to an array
#test_bag_of_words_array = test_bag_of_words_train.toarray()

In [95]:
#test_bag_of_words_array.shape

In [96]:
# predict test using bag of word model
#y_pred_test_bow = bow_model.predict_proba(test_bag_of_words_array)

In [97]:
#y_pred_test_bow[0]

In [98]:
#y_pred_test_df = pd.DataFrame(y_pred_test_bow)

In [99]:
#test_text_df['ID'].shape

In [100]:
#y_pred_test_df['ID']= test_text_df['ID'].reset_index(drop=True)

In [101]:
#y_pred_test_df.head()

In [102]:
# predict test using tfidf model
#y_pred_test_tfidf = tfidf_model.predict_proba(test_bag_of_words_array)

In [103]:
stage2_test_text_df = pd.read_csv('stage2_test_text.csv', sep="\|\|", encoding="utf-8",engine="python", skiprows=1, names=["ID", "Text"])


In [104]:
stage2_test_text_df.shape

(986, 2)

In [105]:
stage2_test_text_df['cleaned_text'] = stage2_test_text_df['Text'].progress_map(cleanse_data,na_action='ignore')

my bar!: 100%|██████████| 986/986 [00:08<00:00, 118.41it/s]


In [106]:
stage2_test_text_df['cleaned_text'] = stage2_test_text_df['cleaned_text'].progress_map(reduce_words)

my bar!: 100%|██████████| 986/986 [00:03<00:00, 265.25it/s]


In [107]:
stage2_test_text_df.shape

(986, 3)

In [108]:
stage2_cleaned_view =pd.DataFrame({"Text":stage2_test_text_df['Text'].progress_map(lambda x:len(x)),"cleaned_text":stage2_test_text_df['cleaned_text'].map(lambda x:len(x))})
stage2_cleaned_view.head()

my bar!: 100%|██████████| 986/986 [00:00<00:00, 246753.21it/s]


,Text,cleaned_text
0,33403,14418
1,66400,29135
2,58544,26808
3,42023,17996
4,22499,8311


In [109]:
stage2_test_corpus = stage2_test_text_df['cleaned_text'].tolist()

In [110]:
#X_stage2_test = bow_vectorizer.fit_transform(stage2_test_corpus).toarray()

In [111]:
X_stage2_test = tfidf_vectorizer.fit_transform(stage2_test_corpus).toarray()

In [112]:
X_stage2_test.shape

(986, 1600)

In [113]:
y_pred_stage2 = tfidf_model.predict_proba(X_stage2_test)

In [114]:
y_pred_stage2.shape

(986, 9)

In [115]:
y_pred_stage2_df = pd.DataFrame(y_pred_stage2)

In [116]:
y_pred_stage2_df['ID']=stage2_test_text_df['ID']

In [117]:
y_pred_stage2_df.columns

Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 'ID'], dtype='object')

In [118]:
y_pred_stage2_df.columns = ['class1','class2','class3','class4','class5','class6','class7','class8','class9','ID']

In [119]:
y_pred_stage2_df=y_pred_stage2_df[['ID','class1','class2','class3','class4','class5','class6','class7','class8','class9']]

In [120]:
y_pred_stage2_df.head()

,ID,class1,class2,class3,class4,class5,class6,class7,class8,class9
0,1,0.186422,0.149421,0.026535,0.237968,0.052618,0.079556,0.249807,0.008294,0.009378
1,2,0.203485,0.151358,0.027601,0.164330,0.064765,0.066240,0.305628,0.007876,0.008715
2,3,0.227005,0.136329,0.028670,0.175229,0.075993,0.056186,0.282988,0.007891,0.009707
3,4,0.178313,0.167293,0.024891,0.173316,0.045918,0.064200,0.331278,0.006635,0.008155
4,5,0.173221,0.178667,0.029656,0.189868,0.055422,0.048550,0.308979,0.007329,0.008308


In [121]:
y_pred_stage2_df.to_csv("y_pred_stage2.csv",index=False)

In [122]:
df=pd.DataFrame(columns=["Gene","Class","Variation","cleaned_text","cleaned_freq"])
for i in range(19):    
    df_new=class_9[class_9['cleaned_text'] == class_9['cleaned_text'].unique()[i]]
    df=pd.concat([df,df_new])

NameError: name 'class_9' is not defined

we have sorted the frequency dictionary based on values using the key parameter of sorted and higher values first using reverse

In [ ]:
sorted(test_dic.items(),key=lambda x:x[1], reverse=True)

In [ ]:
for i in range(10):
    print(scan_data1[i])

In [ ]:
cd /content

In [ ]:
pa_tree=ET.parse('pa2021.xml')

In [ ]:
root=pa_tree.getroot()

In [ ]:
# for child in root:
#   print(child.tag)

In [ ]:
for element_1 in root:
  #print("---------level_1-----------")
  #print(element_1.tag)
  
  for element_2 in element_1:
    #print("---------level_2-----------")
    #print("    "+element_2.tag," ",element_2.text)
    
    for element_3 in element_2:
      #print("---------level_3-----------")
      #print("        "+element_3.tag," ",element_3.text)
      
      for element_4 in element_3:
        #print("---------level_4-----------")
        if element_4.tag=="String":
          print("                   "+element_4.tag," ",element_4.text)
          
  #break
    

In [ ]:
len(root)

In [ ]:
desc_tree=ET.parse('desc2021.xml')

In [ ]:
desc_root=desc_tree.getroot()

In [ ]:
len(desc_root)

In [ ]:
term_description=[]
for element_1 in desc_root:
  #print("-------level_1----------")
  #print(element_1.tag," ",element_1.text)
  for element_2 in element_1:
    #print("    ----------level_2----------")
    #print("    "+element_2.tag," ",element_2.text)
    for element_3 in element_2:
      if element_3.tag=='String':
        #print("        ---------level_3--------")
        #print("        "+element_3.tag," ",element_3.text)
        term_description.append(element_3.text)

  #break

In [ ]:
len(term_description)

In [ ]:
term_description[:10]

In [ ]:
import csv 
with open('term_description.csv', 'w') as f: 
    write = csv.writer(f) 
    write.writerow(term_description) 
    

There are 2 ways to remove punctuation. one is to use the string library punctuation module on the text before tokenization, the other is to use the nltk.punct library

In [ ]:
len(scan_data1[1])

In [ ]:
#nltk.word_tokenize(scan_data1[1])

In [ ]:
# import string punctuation
import string
string.punctuation

In [ ]:
len(cleanup_text(scan_data1[1]))

In [ ]:
scan_data